# Generate a Project Template

## Imports

In [1]:
from typing import Dict, List

import subprocess
import json
import re
from pathlib import Path
from dataclasses import dataclass, InitVar

import pandas as pd


In [2]:
from env_tools import AnacondaException, AbortedCmdException


In [ ]:
from env_tools import console_command, activate_environment


In [3]:
from env_tools import save_env_specs, list_environments

from env_tools import create_environment, remove_environment

from env_tools import install_packages


In [4]:
# %% Initialize logging
import logging  # pylint: disable=wrong-import-position wrong-import-order
logging.basicConfig(level=logging.INFO,
                    format='%(levelname)-8s %(message)s')
logger = logging.getLogger(__name__)
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)


## Utility Functions

In [5]:
def add_folder(base: Path, name: str)->Path:
    '''Make a subdirectory under base.

    Generates a path to the subdirectory.
    If it doesn't exist create it.

    Args:
        base (Path): The top folder (parent to the subdirectory).
        name (str): The name of the subdirectory.

    Returns:
        Path: The path to the subdirectory.
    '''
    sub_folder = base / name
    if not sub_folder.exists():
        sub_folder.mkdir()
    return sub_folder


### Anaconda utilities

## Package Creation

### Create the new environment

In [6]:
new_env = 'Todoist'
python_version = '3.11'


In [11]:
install_output = create_environment(new_env, python_version)


INFO     Creating environment for Todoist


In [12]:
install_output.keys()


dict_keys(['actions', 'prefix', 'success'])

In [13]:
install_output['actions'].keys()


dict_keys(['FETCH', 'LINK', 'PREFIX'])

In [14]:
install_output['actions']['LINK'][0].keys()


dict_keys(['base_url', 'build_number', 'build_string', 'channel', 'dist_name', 'name', 'platform', 'version'])

### Identify the Conda Environment path

In [ ]:
env_path = Path(install_output['prefix'])


In [7]:

delete_output = remove_environment(env_path)


In [ ]:
delete_output.keys()


In [ ]:
delete_output['actions'].keys()


In [ ]:
delete_output['actions']['UNLINK'][0].keys()


### Install the Packages

#### Packages to include in the Conda Environment

In [15]:
primary_package_list = [
    'pysimplegui',
    'pandas',
    'xlwings',
    ]
tools_package_list = [
    'spyder',
    'jupyterlab',
    'ipykernel',
    'pylint',
    ]
package_list = primary_package_list + tools_package_list


In [16]:
install_output_dict = install_packages(new_env, package_list)


In [18]:
install_output_dict.keys()


dict_keys(['actions', 'prefix', 'success'])

In [19]:
install_output_dict['success']


True

In [20]:
install_output_dict['prefix']


'C:\\Users\\smoke\\.conda\\envs\\Todoist'

In [21]:
install_output_dict['actions'].keys()


dict_keys(['FETCH', 'LINK', 'PREFIX'])

In [22]:
install_output_dict['actions']['FETCH'][0].keys()


IndexError: list index out of range

In [23]:
install_output_dict['actions']['FETCH'][0]


IndexError: list index out of range

In [24]:
install_output_dict['actions']['LINK'][0].keys()


dict_keys(['base_url', 'build_number', 'build_string', 'channel', 'dist_name', 'name', 'platform', 'version'])

In [25]:
install_output_dict['actions']['LINK'][0]


{'base_url': 'https://conda.anaconda.org/conda-forge',
 'build_number': 50496,
 'build_string': 'h57928b3_50496',
 'channel': 'conda-forge',
 'dist_name': 'intel-openmp-2023.2.0-h57928b3_50496',
 'name': 'intel-openmp',
 'platform': 'win-64',
 'version': '2023.2.0'}

# Done To Here

`pip install todoist-api-python`

```bat
pip install [options] <requirement specifier> [package-index-options] ...
pip install [options] -r <requirements file> [package-index-options] ...
pip install [options] [-e] <vcs project url> ...
pip install [options] [-e] <local project path> ...
pip install [options] <archive url/path> ...
```
The report can be written to a file, or to standard output (using --report - in combination with --quiet).

`pip freeze > requirements.txt`

### Write the Log Data generated during the Conda Environment install

In [ ]:

log_file = env_folder / f'{new_env}CreationLog.txt'
pkg_table_file = env_folder / f'{new_env}Packages.csv'

install_log = install_log + '\n' + pkg_install_log
log_file.write_text(install_log)

pkg_items = logging_dict['actions']['LINK'] + pkg_install_dict['actions']['LINK']
pkg_table = pd.DataFrame(pkg_items)
pkg_table.to_csv(pkg_table_file)


### Class to hold information on folders related to the package.

In [ ]:
@dataclass
class FolderItem:
    '''Manage folders related to the package.'''
    folder_name: str
    tag: str = ''
    add_to_path: bool = True
    folder_path: Path = None
    base_path: InitVar[Path] = None

    def __post_init__(self, base_path: Path):
        '''Manage defaults for folder settings.

        If not defined, base_path defaults to the current working
        directory.  If it doesn't exists the folder is created.

        If tag is not given use uppercase of folder name for tag.
        '''
        if not base_path:
            base_path = Path.cwd()
        self.set_path(base_path)
        self.set_tag()

    def set_path(self, base_path: Path):
        '''If missing, Set the folder_path to a subdirectory of base_path.

        Generates a path to the subdirectory.
        If it doesn't exist create it.

        Args:
            base_path (Path): The top folder
                (parent to the subdirectory).
        '''
        if not self.folder_path:
            sub_folder = base_path / self.folder_name
            if not sub_folder.exists():
                sub_folder.mkdir()
            self.folder_path = sub_folder

    def set_tag(self):
        '''If tag not given use uppercase of folder name.'''
        if not self.tag:
            self.tag = self.folder_name.upper()


## Project Parameters

### Project Name and Python Version

In [ ]:
new_env = 'Todoist'
python_version = '3.11'


### Packages to include in the Conda Environment

In [ ]:
primary_package_list = [
    'pysimplegui',
    'pandas',
    'xlwings',
    ]
tools_package_list = [
    'spyder',
    'jupyterlab',
    'ipykernel',
    'pylint',
    ]
package_list = primary_package_list + tools_package_list
packages = ' '.join(package_list)


In [ ]:
install_xlwings = True
install_jupyter = True


## Specify project related paths
- The top level project path should already exist and should be given the tag 'WORKSPACE'
- `folder_path` must be specified for 'WORKSPACE'
- An 'ENVIRONMENT' must be defined.
- There shouldn't be duplicate tags.
- If `tag` is not given `folder_name.upper()` will be used as the `tag`
- When `folder_path` is given, the `folder_name` is ignored.  
- If `folder_path` is not given, a folder with the name `folder_name` will be created directly under `base_path`.  
- If `add_to_path` is True *(the default)*, the folder path will be added to the python search path.

#### Example folders:

| folder_name | folder_path                              | tag         | add_to_path |
|-------------|------------------------------------------|-------------|-------------|
| Todoist     | ~\Python\Projects\ToDoIst Projects\Turkey| WORKSPACE   | True        |
| environment | None                                     | ENVIRONMENT | False       |
| src         | None                                     | SOURCE      | True        |
| Old files   | None                                     | IGNORE      | False       |
| data        | None                                     | DATA        | True        |
| reference   | None                                     | REFERENCE   | False       |
| sectionary  | ..\\..\sectionary package\src'           | PACKAGE     | True        |

### Primary project path

In [ ]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")
project_drive = project_folder.drive


### Additional package path

In [ ]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()


#### Folder Parameters

In [ ]:
folder_list = [FolderItem(new_env,       tag='WORKSPACE',                      folder_path=project_folder),
               FolderItem('environment', tag='ENVIRONMENT', add_to_path=False,   base_path=project_folder),
               FolderItem('src',         tag='SOURCE',                           base_path=project_folder),
               FolderItem('data',        tag='DATA',                             base_path=project_folder),
               FolderItem('reference',   tag='REFERENCE',   add_to_path=False,   base_path=project_folder),
               FolderItem('sectionary',                                        folder_path=sectionary_path)]

folders = pd.DataFrame(folder_list)


In [ ]:
folders


In [ ]:
if install_jupyter:


In [ ]:
if install_xlwings:


In [ ]:
print(f'Storing environment for {new_env}')


## Build Boilerplate Files

### Build Jupyter lab Batch file

In [ ]:

jupter_batch_file = env_folder / f'JupyterLab ({new_env}).bat'
jupyter_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    'jupyter-lab'
    ])
jupter_batch_file.write_text(jupyter_batch)


### Build VS Code Batch file

In [ ]:
vscode_batch_file = env_folder / f'VS Code ({new_env}).bat'
vscode_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    f'code {new_env}.code-workspace'
    ])
vscode_batch_file.write_text(vscode_batch)


### Build .env file

In [ ]:
folders


In [ ]:
folders[folders.add_to_path]


In [ ]:
folders[folders.tag=='ENVIRONMENT'].folder_path


In [ ]:
env_file = env_folder / '.env'

env_paths = '\n'.join('PYTHONPATH=${}' for name, path_str in path_dict.items())
path_setter = '\n'.join('PYTHONPATH="${' + name.upper() + '};${PYTHONPATH}"'
                        for name in path_dict.keys())
env_text = '\n'.join([env_paths, path_setter])
env_file.write_text(env_text)


### Build Code-workspace from templatestr(env_path)

In [ ]:
data_dir = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")

worspace_template = data_dir / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [ ]:
worspace = worspace_template.read_text()


In [ ]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }


In [ ]:
str(env_path).replace('\\', '/')


In [ ]:
env_manage_base = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")
code_workspace_template = env_manage_base / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [ ]:
template = code_workspace_template.read_text()


In [ ]:
template = template.replace('%env_path%', str(env_path).replace('\\', '/'))


In [ ]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")

env_folder = add_folder(project_folder, 'environment')
source_folder = add_folder(project_folder, 'src')

project_drive = project_folder.drive


In [ ]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()


In [ ]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }


In [ ]:
template.replace('%analysis.extraPaths%', str(text))


In [ ]:
template.replace('%autoComplete.extraPaths%', str(text))


# Misc Reference

In [ ]:
a = console_command('conda env export --from-history')
print(a)


In [ ]:
new_env = 'Standard'
c = f'cmd /c conda activate {new_env}&&conda env export --from-history'


In [ ]:
c = ''.join([
    r'cmd /c C:\ProgramData\Anaconda3\Scripts\activate.bat ',
    r'C:\ProgramData\Anaconda3',
    r'&&',
    r'conda activate Standard',
    r'&&',
    r'conda env export --from-history'
    ])


In [ ]:
output = subprocess.run(c, shell=True, capture_output=True,
                        check=True)


In [ ]:
print(output.stdout.decode())


In [ ]:
c = ['cmd', r'/c',
     r'C:\ProgramData\Anaconda3\Scripts\activate.bat',
     r'C:\ProgramData\Anaconda3',
     '&&',
     'conda', 'activate Standard',
     '&&',
     'conda', 'env', 'export', '--from-history']


In [ ]:
output = subprocess.run(c, shell=True, capture_output=True,
                        check=True)


In [ ]:
c = ['cmd', r'/c',
     r'C:\ProgramData\Anaconda3\Scripts\activate.bat',
     r'C:\ProgramData\Anaconda3',
     '&&',
     'conda', 'activate Standard',
     '&&',
     'conda', 'env', 'export', '--from-history']
a = console_command(c)
print(a)


## cmd
Starts a new instance of the command interpreter, Cmd.exe. If used without parameters, cmd displays the version and copyright information of the operating system.

#### Syntax
`cmd [/c|/k] [/s] [/q] [/d] [/a|/u] [/t:{<b><f> | <f>}] [/e:{on | off}] [/f:{on | off}] [/v:{on | off}] [<string>]`

#### Parameters

|Parameter|Description|
|-|-|
|/c|Carries out the command specified by <string> and then exits the command processor.|
|/k|Carries out the command specified by <string> and keeps the command processor running.|
|/s|When used with /c or /k, triggers special non-parsing rules that strip the first and last quotes (") around the <string> but leaves the rest of the command unchanged.|
|/q|Turns echo off.|
|/d|Disables execution of AutoRun commands.|
|/a|Formats command output as American National Standards Institute (ANSI).|
|/u|Formats command output as Unicode.|
|/t: {\<b>\<f> \| \<f>}|Sets the background (*b*) and foreground (*f*) colors.|
|/e:on|Enables command extensions.|
|/e:off|Disables commands extensions.|
|/f:on|Enables file and directory name completion.|
|/f:off|Disables file and directory name completion.|
|/v:on|Enables delayed environment variable expansion.|
|/v:off|Disables delayed environment variable expansion.|
|\<string>|Specifies the command you want to carry out.|
|/?|Displays help at the command prompt.|

The following table lists valid hexadecimal digits that you can use as the values for \<b> and \<f>:

|Value|Color|
|-|-|
|0|Black|
|1|Blue|
|2|Green|
|3|Aqua|
|4|Red|
|5|Purple|
|6|Yellow|
|7|White|
|8|Gray|
|9|Light blue|
|a|Light green|
|b|Light aqua|
|c|Light red|
|d|Light purple|
|e|Light yellow|
|f|Bright white

#### Remarks
- To redirect command output to the input of another command, use the pipe (`|`) character. For example:
> `<command1> | <command2>`

- To redirect command output to a file, use the greater-than angle bracket `>` character. For example:

`<command1> > <file1.txt>`
To use multiple commands for `<string>`, separate them by the command separator `&&`. For example:
> `<command1>&&<command2>&&<command3>`

- If the directory path, files, or any information you supply contains spaces, you must use double quotation marks (`"``) around the text, such as `"Computer Name"`. For example:
> `mkdir Test&&mkdir "Test 2"&&move "Test 2" Test`
> 
- You must use quotation marks around the following special characters: & \< > [ ] | { } ^ = ; ! ' + , ` ~ [white space].

- If you specify `/c` or `/k`, the `cmd` processes, the remainder of `<string>`, and the quotation marks are preserved only if all of the following conditions are met:
   - You don't also use `/s`.
   - You use exactly one set of quotation marks.
   - You don't use any special characters within the quotation marks (for example: & <    ( ) @ ^ | ).
   - You use one or more white-space characters within the quotation marks.
   - The `<string>` within quotation marks is the name of an executable file.

If the previous conditions aren't met, `<string>` is processed by examining the first character to verify whether it's an opening quotation mark. If the first character is an opening quotation mark, it's stripped along with the closing quotation mark. Any text following the closing quotation marks is preserved.

## Common Packages
> **Primary Packages**
> - numpy
> - pandas
> - openpyxl
> - matplotlib
> - seaborn
> - pysimplegui

> **iPython Packages**
> - ipykernel
> - ipywidgets
> - jupyterlab
> - nb_conda_kernels
> - spyder

> **SciPy Packages**
> - scipy
> - scikit-image
> 
> **Specialty Packages**
> - networkx
> - pygraphviz
> - shapely
> - pydicom
> - pyodbc
> - parsedatetime

> **MS Office & Windows related Packages**
> - xlwings
> - pywin32
> - docx2txt
> - zipfile2

> **Linting Packages**
> - pylint
> - mypy
> - black

> **Installer**
> - pyinstaller

> **Documentation Packages**
> - sphinx
> - sphinx-copybutton
> - sphinx_rtd_theme
> - nbsphinx-link
> - nbsphinx
> - esbonio
> - pandoc
> - myst-parser

> **AI Related Packages**
> - scikit-learn
> - neptune-client
> - neptune-sklearn
> - statsmodels
> - xgboost
> - torchvision
> - nibabel
> - nilearn


### Other Install Commands

#### Install Jupyter Kernel 

In [ ]:
def install_kernel(new_env):
    '''Install the ipython kernel in the new environment.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    kernel_cmd = activate_environment(new_env)
    kernel_cmd += f'python -m ipykernel install --user --name {new_env} '
    kernel_cmd += f'--display-name "Python ({new_env})"'

    kernel_install_log = console_command(
        kernel_cmd,
        AnacondaException,
        f'Unable to install ipython kernel in "{new_env}"!'
        )
    return kernel_install_log


#### Configure xlwings

In [ ]:
def configure_xlwings(new_env: str)->str:
    '''Install and configure xlwings binaries for environment.

    NOTE: This is still a works in progress.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    activate_cmd = activate_environment(new_env)

    logger.debug('Updating Xlwings licence for %s', new_env)
    xlwings_license = activate_cmd + 'xlwings license update -k noncommercial'
    license_log = console_command(
        xlwings_license, AnacondaException,
        f'Unable to set xlwings license in environment {new_env}')

    logger.debug('Installing Xlwings add-in for %s', new_env)
    xlwings_add_in = activate_cmd + 'xlwings addin install'
    try:
        add_in_log = console_command(
            xlwings_add_in, AnacondaException,
            f'Unable to install xlwings addin in environment {new_env}',
            abort_after=2)
    except AbortedCmdException:
        logger.warning('Xlwings Add-in install timed out\n'
                       'This is likely due to insufficient rights to install '
                       'the add-in.')

    logger.debug('Creating Xlwings config file for %s', new_env)
    xlwings_config = activate_cmd + 'xlwings config create --force'
    config_log = console_command(
        xlwings_config, AnacondaException,
        f'Unable to configure xlwings in environment {new_env}')

    xlwings_log = '\n\n'.join([license_log, add_in_log, config_log])
    return xlwings_log


***The commands below hang the kernel***

In [ ]:
new_env = 'Todoist'
activate_cmd = activate_environment(new_env)

xlwings_license = activate_cmd + 'xlwings license update -k noncommercial'
license_log = console_command(
    xlwings_license, AnacondaException,
    f'Unable to set xlwings license in environment {new_env}')

print(license_log)


In [ ]:
cmd_str = activate_environment('Standard')
cmd_str += 'conda env export --from-history'
output = console_command(cmd_str)
print(output)


In [ ]:
output = install_kernel('Todoist')
print(output)
